In [1]:
import pandas as pd
import numpy as np
import pickle
from constraints import generate_constraints

import xgboost as xgb
from HopSkipJump_Tabular import HopSkipJump
from Boundary_Tabular import BoundaryAttack
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
# wrappers
from art.estimators.classification import XGBoostClassifier, SklearnClassifier, xgboost



c:\Users\Administrator\.conda\envs\ppoEnv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Generate random tabular dataset

In [2]:
tabular_dataset = pd.DataFrame(np.random.uniform(-50, 50, size=(100, 10)), columns=list('ABCDEFGHIJ'))
tabular_dataset['pred'] = np.random.randint(0,2,size=(100, 1))
tabular_dataset.head()

,A,B,C,D,E,F,G,H,I,J,pred
0,8.737470,-4.452732,-42.402433,13.297259,-11.634207,46.186411,40.225368,-12.961446,-35.908975,31.561008,0
1,-44.276731,2.636129,30.385704,24.032581,-32.991834,-28.681431,-0.575582,4.045955,10.718274,14.219855,1
2,-25.251271,0.371304,-1.059176,-42.067652,0.764938,-1.374950,-14.131471,-42.963698,21.557089,47.554996,1
3,-28.832128,-5.297842,27.231204,-26.490244,-33.991947,42.687121,-29.711690,6.580710,-14.711784,16.963742,1
4,-7.901436,5.544106,-15.027453,-13.639961,-10.767784,-40.407575,-23.900719,-31.523658,-2.486473,37.164518,0


Training classification model (XGB)

In [3]:
# Separate features (X) and target label (y)
X = tabular_dataset.drop('pred', axis=1)
y = tabular_dataset['pred']

# Split into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize XGBoost classifier
model = xgb.XGBClassifier(use_label_encoder=False, eval_metric='logloss')

# Train the model
model.fit(X_train, y_train)

# Make predictions and evaluate
y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)

print(f"Model accuracy: {accuracy * 100:.2f}%")

Model accuracy: 45.00%


c:\Users\Administrator\.conda\envs\ppoEnv\lib\site-packages\xgboost\core.py:158: UserWarning: [21:50:18] WARNING: D:\bld\xgboost-split_1724807710452\work\src\learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


Attacks parameters

In [4]:
columns_names = list(X_test.columns)
attack_x = np.array(X_test.copy())
true_label = y_test.transpose().values.tolist()[0]


In [8]:
### boundary attack params ###
epsilon=1.
delta = 1.
max_iter_b =3000
num_trial =2000
adapt_step = 1.
boundary_params = {'epsilon':epsilon, 'delta':delta, 'max_iter':max_iter_b, 'num_trial':num_trial, 'step_adapt':adapt_step}

### hopskip attack params ###
batch_size = 128
norm = 2
max_iter_h = 500
max_eval_h = 1000
init_eval = 500
init_size = 1000
hopskip_params = {'batch_size':batch_size, 'norm':norm, 'init_eval':init_eval, 'init_size':init_size, 'max_iter':max_iter_h, 'max_eval':max_eval_h}


generate_params = {'dataset':tabular_dataset, 'mask':None, 'columns_names':columns_names, 'seed':12}

HopSkipJump attack generation and evaluation

In [6]:
# Wrapper for ART
target_model = XGBoostClassifier(model=model, nb_features=attack_x.shape[1], nb_classes=2)

hopSkip_attack = HopSkipJump(classifier=target_model, **hopskip_params)
adv, queries, failed_ind = hopSkip_attack.generate( x=attack_x, **generate_params, x_adv_init=None)

## EVALUATE ATTACK ###
        
pred_original_target = np.argmax(target_model.predict(X_test), axis=1)
pred_adv_target = np.argmax(target_model.predict(adv), axis=1)

### -------- compute success -------- ###
target_success = pred_adv_target.flatten() != pred_original_target.flatten()

success =  100 * np.sum(target_success)/ attack_x.shape[0]
print('Success rate of HopSkipJump attack: {:.2f}'.format(success))

HopSkipJump: 100%|██████████| 20/20 [01:15<00:00,  3.77s/it]

Success rate of HopSkipJump attack: 100.00


Boundary attack generation and evaluation

In [9]:
# Wrapper for ART
target_model = XGBoostClassifier(model=model, nb_features=attack_x.shape[1], nb_classes=2)

boundary_attack = BoundaryAttack( estimator=target_model, **boundary_params)
adv, queries, failed_ind = boundary_attack.generate( x=attack_x, **generate_params, x_adv_init=None)

## EVALUATE ATTACK ###
        
pred_original_target = np.argmax(target_model.predict(X_test), axis=1)
pred_adv_target = np.argmax(target_model.predict(adv), axis=1)

### -------- compute success -------- ###
target_success = pred_adv_target.flatten() != pred_original_target.flatten()

success =  100 * np.sum(target_success)/ attack_x.shape[0]
print('Success rate of Boundary attack: {:.2f}'.format(success))

Boundary attack: 100%|██████████| 20/20 [02:19<00:00,  6.96s/it]

Success rate of Boundary attack: 100.00
